For this exercise, we will use the "Internet Advertisements" data set from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Internet+Advertisements). 

This dataset represents a set of possible advertisements on Internet pages. The features encode the geometry of the image (if available) as well as phrases occuring in the URL, the image's URL and alt text, the anchor text, and words occuring near the anchor text. The task is to predict whether an image is an advertisement ("ad") or not ("nonad").

### Import packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, roc_auc_score

%matplotlib inline

### Set-up

In [ ]:
# location of the output files
path = r'C:\Users\jonathan\Desktop'

data_file = path + r'\ad.data'
header_file = path + r'\ad.names'

### Read data

In [ ]:
df = pd.read_csv(data_file, header=None)

df.head()

In [ ]:
df.shape

Read header.

In [ ]:
header_text = open(header_file, "r")

lines = header_text.read().split('\n')

lines[:20]

In [ ]:
header = []

for line in lines[4:-1]:
    
    if line[0] not in ('|', ''):
        header.append(line.split(':')[0])
        
header[:20]

In [ ]:
header.append('ad')

target = 'ad'

df.columns = header

df.head()

In [ ]:
df[target].value_counts()

Convert the target variables from strict to numeric (binary).

In [ ]:
df[target] = [1 if x == 'ad.' else 0 for x in df[target]]

df[target].value_counts() / len(df)

In [ ]:
df.dtypes

In [ ]:
df.height.value_counts()[:10]

Convert the four continuous variables into numeric (float). The code below will replace all '?' with nulls.

In [ ]:
df = df.apply(pd.to_numeric, errors='coerce')

df.dtypes

### Set-up X and y

In [ ]:
X = df.drop(target, axis=1)

all_vars = X.columns 

y = df[target]

X.shape, y.shape

### Descriptive statistics

In [ ]:
desc = X.describe(percentiles=[.98, .99])

desc.head()

In [ ]:
desc = desc.T

desc.head()

Missing values

In [ ]:
desc['pct_missing'] = 1 - desc['count'] / len(X)

Kurtosis

In [ ]:
desc['kurtosis'] = X.kurtosis()

Correlation with the target

In [ ]:
all_corr, all_corr_abs = [], []

for var in all_vars:
    
    all_corr.append(X[var].corr(y))
    all_corr_abs.append(abs(X[var].corr(y)))

In [ ]:
desc['corr'] = all_corr

desc['abs_corr'] = all_corr_abs

In [ ]:
desc.to_csv(path + r'\desc.csv')

### Trimming

There are two (non-binary) variables with kurtosis 10 or higher.

In [ ]:
trim_vars = ['height', 'aratio']

for trim_var in trim_vars:
    
    print (trim_var)    
    p99 = desc.loc[trim_var, '99%']
    
    X[trim_var] = [min(x, p99) for x in X[trim_var]]
    
    print ('Kurtosis --> before', desc.loc[trim_var, 'kurtosis'], 'after:', X[trim_var].kurtosis()) 

In [ ]:
trim_vars = ['aratio']

for trim_var in trim_vars:
    
    p98 = desc.loc[trim_var, '98%']
    
    X[trim_var] = [min(x, p98) for x in X[trim_var]]
    
    print ('Kurtosis --> before', desc.loc[trim_var, 'kurtosis'], 'after:', X[trim_var].kurtosis()) 

### Missing Value Imputation

In [ ]:
impute_vars = ['height', 'width', 'aratio', 'local']

for impute_var in impute_vars:
    
    # Create missing value indicator
    X[impute_var + '_M'] = [1 if pd.isnull(x) else 0 for x in X[impute_var]]
    
    median = desc.loc[impute_var, '50%']
    
    X[impute_var] = X[impute_var].fillna(median)
    
X[impute_vars].describe().T

In [ ]:
for impute_var in impute_vars:
    print (impute_var, X[impute_var].corr(y))

### Transformations

In [ ]:
xform_vars = ['height', 'width', 'aratio']

# For this exercise, we will try log transformation

for xform_var in xform_vars:
    
    print (xform_var)
    
    log_x = np.log(X[xform_var])
    
    orig_abs_corr = desc.loc[impute_var, 'abs_corr']
    
    new_abs_corr = abs(y.corr(log_x))
    
    corr_pct_improvement = (new_abs_corr - orig_abs_corr) / orig_abs_corr
    
    print ('Original: ', orig_abs_corr, 'New: ', new_abs_corr, 'Improvement: ', corr_pct_improvement)
    
    if corr_pct_improvement >= 0.05:
        
        X[xform_var + '_log'] = [np.log(x) for x in X[xform_var]]
        
        X.drop(xform_var, axis=1, inplace=True)

### Standard Deviations

In [ ]:
sns.set_style('darkgrid')

plt.subplots(figsize = (12, 9))

plt.plot(range(len(desc)), desc['std'])

plt.xlabel('Attributes', fontsize=14)
plt.ylabel('Standard Deviation', fontsize=14);

In [ ]:
plt.subplots(figsize = (12, 9))

plt.plot(range(len(desc[desc['max']==1])), sorted(desc[desc['max']==1]['std'], reverse=True))

plt.xlabel('Binary Attributes', fontsize=14)
plt.ylabel('Standard Deviation', fontsize=14);

In [ ]:
plt.subplots(figsize = (12, 9))

plt.plot(range(len(desc[desc['std']<.1])), sorted(desc[desc['std']<.1]['std'], reverse=True))

plt.xlabel('Binary Attributes', fontsize=14)
plt.ylabel('Standard Deviation', fontsize=14);

In [ ]:
desc = X.describe().T

core_vars = desc.index[desc['std'] > 0.055]

len(core_vars)

### Correlation with the Target

In [ ]:
all_corr_abs = {}

for var in core_vars:

    all_corr_abs[var] = abs(X[var].corr(y))
    
all_corr_abs_df = pd.DataFrame.from_dict([all_corr_abs]).T

all_corr_abs_df.columns=['abs_corr']

all_corr_abs_df['rank'] = all_corr_abs_df.rank(ascending=False) 

all_corr_abs_df = all_corr_abs_df.sort_values(by='rank')

all_corr_abs_df.head()

In [ ]:
plt.subplots(figsize = (12, 9))

plt.plot(all_corr_abs_df['rank'], all_corr_abs_df['abs_corr'])

plt.xlabel('Attribute Rank', fontsize=14)
plt.ylabel('Correlation with Target', fontsize=14);

Since we will not be performing multi-collinearity analysis in this exercise to further reduce the number of features, we will use a more aggressive threshold to drop variables that have a low correlation coefficient with the target. The first elbow occurs around 0.15; we will use this cutoff value to reduce the number of features.

In [ ]:
corr_tol = 0.15

core_vars = all_corr_abs_df.index[all_corr_abs_df['abs_corr'] >= corr_tol]

X_core = X[core_vars]

X_core.shape

### Tri-fold Partitioning

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_core, y, test_size=2000, random_state=314)

X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=1000, random_state=314)

len(X_train), len(X_test), len(X_valid)

### Standardize

In [ ]:
X_scaler = StandardScaler()

X_train_scaled = X_scaler.fit_transform(X_train.astype(float))
X_test_scaled = X_scaler.transform(X_test.astype(float))
X_valid_scaled = X_scaler.transform(X_valid.astype(float))

### Logistic Regression

Let's first assign a rank to each variable.

In [ ]:
logit = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=314)

rfe = RFE(estimator=logit, n_features_to_select=1)

rfe.fit(X_train_scaled, y_train)

ranking = rfe.ranking_

ranking[:10]

In [ ]:
sorted(zip(rfe.ranking_, core_vars))[:15]

In [ ]:
sorted_vars = sorted(zip(rfe.ranking_, core_vars))

len(sorted_vars)

In [ ]:
max_vars = 25
auc_train_all, auc_test_all = [], []

for num_preds in range(max_vars):
    
    ranked_preds = sorted_vars[:num_preds+1]
    
    preds = [x[1] for x in ranked_preds]
    #print (preds)
    
    _X_train_scaled = X_scaler.fit_transform(X_train[preds].astype(float))
    _X_test_scaled = X_scaler.transform(X_test[preds].astype(float))

    logit.fit(_X_train_scaled, y_train)
    
    logit_scores_train = logit.predict_proba(_X_train_scaled)[:, 1]
    logit_scores_test = logit.predict_proba(_X_test_scaled)[:, 1]
    
    auc_train = roc_auc_score(y_train, logit_scores_train)
    auc_test = roc_auc_score(y_test, logit_scores_test)
    
    auc_train_all.append(auc_train)
    auc_test_all.append(auc_test)    

In [ ]:
plt.subplots(figsize = (12, 9))

plt.plot(range(max_vars), auc_train_all, lw=2, label='Train')
plt.plot(range(max_vars), auc_test_all, lw=2, label='Test')

plt.xlabel('Number of Predictors', fontsize=14)
plt.ylabel('AUC', fontsize=14)
plt.legend();

In [ ]:
sns.set_style('dark')

diff = [a - b for a, b in zip(auc_train_all, auc_test_all)]

f, ax = plt.subplots(figsize = (12, 9))
ax2 = ax.twinx()

ax.plot(range(max_vars), auc_train_all, lw=2, label='Train', linestyle = '--', color='royalblue')
ax.plot(range(max_vars), auc_test_all, lw=2, label='Test', color='royalblue')

ax2.plot(range(max_vars), diff, lw=1, color = 'orangered', label = 'Difference')

ax.plot([13, 13], [0.7, 1], color='gray', lw=.5)

ax.set_xlabel('Number of Predictors', fontsize = 14)
ax.set_ylabel('AUC', fontsize = 14)
ax2.set_ylabel('Difference in AUC', fontsize = 14)
ax.set_ylim([.7, 1]);

In [ ]:
ranked_preds = sorted_vars[:13]
    
final_preds = [x[1] for x in ranked_preds] 

len(final_preds)

In [ ]:
final_preds

### Final Model

In [ ]:
# We can combine Train and Test sets to build the final model

X_train_test = X_train.append(X_test)
y_train_test = y_train.append(y_test)

X_train_test.shape, y_train_test.shape

In [ ]:
X_train_scaled = X_scaler.fit_transform(X_train_test[final_preds].astype(float))
X_valid_scaled = X_scaler.transform(X_valid[final_preds].astype(float))

logit.fit(X_train_scaled, y_train_test)

logit_scores_train = logit.predict_proba(X_train_scaled)[:, 1]
logit_scores_valid = logit.predict_proba(X_valid_scaled)[:, 1]

auc_train = roc_auc_score(y_train_test, logit_scores_train)
auc_valid = roc_auc_score(y_valid, logit_scores_valid)   

print(auc_train, auc_valid)

### Model Details

In [ ]:
logit.coef_

In [ ]:
logit.intercept_

Create a dataframe to save the model coefficients.

In [ ]:
model_pred_coeff = pd.DataFrame(list(zip(final_preds, [x[0] for x in np.transpose(logit.coef_)])))

model_pred_coeff.columns=[['Predictor', 'Coefficient']]

model_pred_coeff

In [ ]:
model_pred_coeff.loc[13]= ['Intercept', logit.intercept_[0]]

model_pred_coeff

In [ ]:
model_pred_coeff.to_csv(path + r'model_coeff.csv')

### ROC Curve

In [ ]:
sns.set_style('darkgrid')

# Calculate the false positive and true positive rates
logit_fpr_train, logit_tpr_train, _ = roc_curve(y_train_test, logit_scores_train)
logit_fpr_valid, logit_tpr_valid, _ = roc_curve(y_valid, logit_scores_valid)

plt.figure().set_size_inches(12, 9)

plt.plot(logit_fpr_train, logit_tpr_train, c='royalblue', lw=2, alpha=0.3, linestyle = '--',
         label='Training (AUC = %0.3f)' %auc_train)

plt.plot(logit_fpr_valid, logit_tpr_valid, c='royalblue', lw=2, 
         label='Validation (AUC = %0.3f)' %auc_valid)

plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='-', alpha=.5)
plt.xlabel('False Positive Rate', fontsize = 14)
plt.ylabel('True Positive Rate', fontsize = 14)
plt.title('Ad Identification Model', fontsize = 16)
plt.legend(loc="lower right", fontsize = 14);

In [ ]:
preds_valid = logit.predict(X_valid_scaled)

pd.crosstab(y_valid, preds_valid)

In [ ]:
confusion_matrix = pd.crosstab(y_valid, preds_valid) / len(y_valid)

confusion_matrix

In [ ]:
np.trace(confusion_matrix)

In [ ]:
sns.set_style('darkgrid')

plt.figure().set_size_inches(12, 9)

sns.distplot(logit_scores_valid);

### Mode Lift and Gains

In [ ]:
logit_scores_valid[:10]

In [ ]:
lift_df = pd.DataFrame(data=logit_scores_valid, columns=['score'])

lift_df['y'] = y_valid.values

lift_df = lift_df.sort_values(by='score', ascending=False)

lift_df.head()

In [ ]:
lift_df['cumul_y'] = lift_df['y'].cumsum()

lift_df['cumul_pctg_y'] = lift_df['cumul_y'] / sum(lift_df['y'])

lift_df.tail(10)

In [ ]:
lift_df = lift_df.reset_index()

lift_df.head()

In [ ]:
lift_df['rank'] = lift_df.index + 1

lift_df['rank'].head()

In [ ]:
lift_df['decile'] = pd.qcut(lift_df['rank'], 10, labels=False)

lift_df['decile'].value_counts()

In [ ]:
lift_df.groupby('decile')['y'].mean()

Calculate Model Lift by decile.

In [ ]:
lift_df.groupby('decile')['y'].mean() / lift_df['y'].mean()

Calculate Model Gains by decile

In [ ]:
y_sum = lift_df.groupby('decile')['y'].sum().cumsum() / lift_df['y'].sum()

y_sum

### Predictor Profiles

In [ ]:
binary_preds = [x for x in final_preds if x != 'width_log']

In [ ]:
sns.set_style('white')

for binary_pred in binary_preds:
    
    attr_values = df[binary_pred]
    
    y_rates = df.groupby([binary_pred])[target].mean().reset_index()
    x_pctg = df[binary_pred].value_counts() / len(df)
    
    fig = plt.figure()
    ax = x_pctg.plot(kind='bar', color='royalblue', alpha=.5)
    ax2 = ax.twinx()
    ax2.plot(['N' if x==0 else 'Y' for x in y_rates[binary_pred]], y_rates[target], lw=3, color = 'tomato')
    
    plt.title(binary_pred, fontsize=14)
    ax.set_xlabel(target, fontsize = 14)
    ax.xaxis.set_tick_params(rotation=0)
    ax.set_ylabel('% of Records', fontsize = 14)
    ax2.set_ylabel('Target Rate', fontsize = 14);

For the continous predictor, you can first create bins and then create the profile plots. I will leave this as an exercise for you.

### Save the scored dataset

In [ ]:
X[final_preds].shape

In [ ]:
X_scaled = X_scaler.transform(X[final_preds].astype(float))

X['Predicted Probability'] = logit.predict_proba(X_scaled)[:, 1]
X['Predicted Class'] = logit.predict(X_scaled)

X.head()

In [ ]:
X['Predicted Probability'].describe()

In [ ]:
X.to_csv(path + r'\scored_df.csv')

### Predictor means and standard deviations

In [ ]:
pred_stats = pd.DataFrame(list(zip(X_train[final_preds].mean(), X_train[final_preds].std())), 
                          index=[final_preds],
                          columns=['mean', 'stdev'])

pred_stats

### Prep for the scoring process

In [ ]:
for i, rec in pred_stats.iterrows():
    pred = i[0]
    mean = rec['mean']
    stdev = rec['stdev']
    print ("df['%s']" %pred, '= [(x - ', mean, ') /', stdev, "for x in df['%s']" %pred, ']')

In [ ]:
for i, rec in model_pred_coeff.iterrows():
    
    pred = rec['Predictor'][0]
    coeff = rec['Coefficient'][0]
    
    if pred != 'Intercept':
            print ("df['"+pred+"'] *", coeff, '+')
    else:
        print (coeff)
    